In [1]:
import pandas as pd
from fastbook import *
from fastai.tabular.all import *

c:\Users\oirys\miniforge3\envs\DISCOV_env\lib\site-packages\fastbook\__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
c:\Users\oirys\miniforge3\envs\DISCOV_env\lib\site-packages\fastbook\__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


In [12]:
df_nn = pd.read_csv('../Data/data_for_machine_learning.csv', low_memory=False,sep=";")

output_name = '../models/CIAO_ML.pkl'

# Training a new version of DISCOV using a CSV

In [13]:
df_nn

,log(chl),log(rrs443),log(rrs490),log(rrs510),log(rrs560),DOY
0,0.482159,-2.775414,-2.804895,-2.791867,-2.902937,103
1,0.802089,-2.999033,-2.934175,-2.883550,-2.888819,100
2,-0.663540,-2.377580,-2.381586,-2.450495,-2.643155,209
3,-0.185087,-1.992210,-1.986268,-2.031670,-2.203480,230
4,-0.481486,-2.360939,-2.423062,-2.513627,-2.761145,206
...,...,...,...,...,...,...
1367,-0.104001,-2.189091,-2.180157,-2.233240,-2.369714,205
1368,-0.151601,-2.614846,-2.540348,-2.513737,-2.592851,187
1369,-0.223502,-2.517196,-2.493221,-2.540484,-2.727258,223
1370,0.323829,-2.606952,-2.466715,-2.428564,-2.425812,171


In [24]:
dep_var = "log(chl)"
splits = RandomSplitter()(df_nn)

In [15]:
cont_nn,cat_nn = cont_cat_split(df_nn, dep_var=dep_var)

In [28]:
cont_nn

['log(rrs443)', 'log(rrs490)', 'log(rrs510)', 'log(rrs560)', 'DOY']

In [29]:
procs_nn = [FillMissing]
to_nn = TabularPandas(df_nn, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var, y_block=CategoryBlock())

In [30]:
to_nn

      log(chl)  log(rrs443)  log(rrs490)  log(rrs510)  log(rrs560)  DOY
750        283    -2.665642    -2.643342    -2.687590    -2.852788  259
981         71    -2.261597    -2.319170    -2.416547    -2.675847  236
767         21    -2.386827    -2.451101    -2.556581    -2.820552  200
953        724    -2.184809    -2.146111    -2.160665    -2.234959  239
1072       388    -2.378270    -2.380606    -2.424053    -2.601914  233
...        ...          ...          ...          ...          ...  ...
907         -1    -2.539555    -2.604635    -2.652573    -2.847762  141
584        554    -2.335971    -2.362725    -2.421612    -2.589659  196
1239        -1    -2.671718    -2.538347    -2.484748    -2.504875  173
154         -1    -2.755910    -2.696705    -2.695861    -2.783264  233
325         -1    -2.292266    -2.379334    -2.490604    -2.772734  214

[1372 rows x 6 columns]

In [33]:
dls = to_nn.dataloaders()
dls.valid.show_batch()

,log(rrs443),log(rrs490),log(rrs510),log(rrs560),DOY,log(chl)
0,-2.382591,-2.433126,-2.522269,-2.771486,197.0,-0.866461
1,-2.354681,-2.398660,-2.496270,-2.726826,189.0,NaN
2,-2.487198,-2.464542,-2.438660,-2.459125,245.0,NaN
3,-2.609224,-2.681685,-2.752170,-2.988935,147.0,NaN
4,-2.326101,-2.432292,-2.553804,-2.841383,224.0,NaN
5,-2.313586,-2.366061,-2.471215,-2.721719,189.0,NaN
6,-2.072592,-2.177456,-2.344630,-2.693048,227.0,-0.982967
7,-2.469627,-2.439579,-2.464190,-2.608014,156.0,NaN
8,-2.249311,-2.334307,-2.454930,-2.744260,191.0,NaN
9,-2.341598,-2.368844,-2.465036,-2.703669,186.0,NaN


In [20]:
y = to_nn.train.y
y.min(),y.max()

(0, 978)

In [12]:
#dls.device=device

In [21]:
learn = tabular_learner(dls,n_out=10)

In [22]:
learn.lr_find()

IndexError: Target 473 is out of bounds.

In [21]:
learn.summary()

TabularModel (Input shape: 64 x 0)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 22             
BatchNorm1d                               44         True      
____________________________________________________________________________
                     64 x 200            
Linear                                    4400       True      
ReLU                                                           
BatchNorm1d                               400        True      
____________________________________________________________________________
                     64 x 100            
Linear                                    20000      True      
ReLU                                                           
BatchNorm1d                               200        True      
____________________________________________________________________________
                     64 x 10             
Linear                                    1010       T

In [23]:
learn.fine_tune(12)

epoch,train_loss,valid_loss,time
0,0.077760,0.065612,02:08


epoch,train_loss,valid_loss,time
0,0.069165,0.057546,02:07


In [24]:
preds,targs = learn.get_preds()
preds,targs

(tensor([[7.6056e-01, 4.7421e-07, 1.1563e-04,  ..., 3.6796e-07, 8.9293e-08, 9.3938e-08],
         [6.6790e-04, 2.0776e-08, 1.8779e-06,  ..., 9.9566e-07, 4.6773e-08, 4.3127e-08],
         [2.9396e-01, 1.1964e-05, 4.2816e-04,  ..., 2.0045e-06, 4.8907e-07, 4.7378e-07],
         ...,
         [1.3663e-06, 4.6333e-04, 9.9755e-01,  ..., 5.5767e-09, 1.2119e-07, 1.1210e-07],
         [7.7549e-01, 2.5260e-06, 5.9374e-05,  ..., 3.4678e-06, 1.4651e-07, 1.5162e-07],
         [1.0409e-03, 1.8561e-07, 4.2232e-06,  ..., 3.8318e-08, 5.3561e-08, 5.2309e-08]]),
 tensor([[0],
         [5],
         [0],
         ...,
         [2],
         [0],
         [5]], dtype=torch.int8))

In [25]:
error_rate(preds, targs)

TensorBase(0.0206)

In [26]:
learn.export(output_name)